# PUBLIC 20위 코드

# Deep Learning

In [1]:
# 필요한 패키지 IMOPRT
from tensorflow.python import keras
from tensorflow.python.keras import backend as K
from tensorflow.python.keras import regularizers
from tensorflow.python.keras.optimizers import Adam,RMSprop,Nadam
from tensorflow.python.keras.models import Sequential, Model, load_model
from tensorflow.python.keras.layers import Conv2D, Conv2DTranspose, Activation, Flatten, Dense, UpSampling2D, Reshape, Lambda,LSTM,Bidirectional, Permute,Multiply,Add,Concatenate,Dot
from tensorflow.python.keras.layers import Input,BatchNormalization, Dropout, LeakyReLU,MaxPooling2D,Embedding,Conv1D,GlobalMaxPooling1D,MaxPooling1D,GaussianNoise,GaussianDropout,AlphaDropout
from tensorflow.python.keras.losses import mean_absolute_error,mean_squared_error
from tensorflow.keras.utils import plot_model
from tensorflow.python.keras.callbacks import EarlyStopping, LearningRateScheduler
import pandas as pd
import warnings
import numpy as np 
#from pandas_profiling import ProfileReport
from keras.layers import Activation
from keras.utils.generic_utils import get_custom_objects

Using TensorFlow backend.


In [2]:
# Train 데이터를 불러옵니다.
train = pd.read_csv('../data/train.csv')
# Test 데이터를 불러옵니다.
test = pd.read_csv('../data/test.csv')

In [ ]:
# 피쳐 LIST를 만듭니다.
feature_list=train.columns.to_list()
feature_list.remove('id')
feature_list.remove('type')
feature_list.remove('fiberID')

In [ ]:
unique_labels = train['type'].unique()
label_dict = {val: i for i, val in enumerate(unique_labels)}
i2lb = {v:k for k, v in label_dict.items()}

In [ ]:
train_X = train.iloc[:,3:]  #Train 데이터의 독립변수
train_X_type=train.iloc[:,2]
train_Y = train.iloc[:,1] #Train 데이터의 종속변수
test_X = test.iloc[:,2:]    #Test  데이터의 독립변수
test_X_type=test.iloc[:,1]
test_ids = test['id']
train_Y = train_Y.replace(label_dict)

In [ ]:
# 스케일링을 합니다.
from sklearn.preprocessing import MinMaxScaler,StandardScaler,QuantileTransformer,RobustScaler
scaler = RobustScaler()
scaler.fit(train_X)
train_X=scaler.transform(train_X)
test_X=scaler.transform(test_X)

In [ ]:
train_X=pd.DataFrame(train_X)
train_Y=pd.DataFrame(train_Y)

In [ ]:
from sklearn.model_selection import StratifiedKFold , train_test_split , KFold
t_x,v_x,t_x_type,v_x_type,t_y,v_y = train_test_split(train_X,train_X_type,train_Y , stratify = train_Y , random_state = 730, test_size=0.2)

In [ ]:
def gelu(x):
    return 0.5*x*(1+K.tanh(np.sqrt(2/np.pi)*(x+0.044715*K.pow(x, 3))))

In [ ]:
es = EarlyStopping(monitor='val_loss',patience=10)

In [ ]:
def hjk_sch(epoch):
    init_lr=0.003
    min_lr=0.00003
    e=150
    step=np.exp(np.log(min_lr/init_lr)/e)
    if(epoch<20):
        return init_lr
    else :
        hjk_lr=init_lr*step**(epoch-20)
        if(hjk_lr>min_lr):
            return hjk_lr
        else:
            return min_lr

hjk_scher = keras.callbacks.LearningRateScheduler(hjk_sch)

In [ ]:
input_num=20
input_type_num=1000
output_num=19

i=Input(shape=(input_num,))
i_type=Input(shape=(1,))
out_type=Embedding(input_type_num,5)(i_type)
out_type=Reshape((5,))(out_type)
out=Concatenate()([i,out_type])
out=Reshape((5,5))(out)
out=LSTM(input_num,return_sequences=True)(out)
out=Permute((2,1))(out)
out=LSTM(input_num,return_sequences=True)(out)
out=Permute((2,1))(out)
out=Flatten()(out)
out=Dense(output_num*20,activation=gelu)(out)
out=Dense(output_num*15,activation=gelu)(out)
out=Dense(output_num*10,activation=gelu)(out)
out=Dense(output_num*5,activation=gelu)(out)
out=Dense(output_num,activation='softmax')(out)
hjk_DL_model=Model(inputs=[i,i_type],outputs=[out])

In [ ]:
plot_model(hjk_DL_model,show_shapes=True)

In [ ]:
hjk_DL_model.compile(loss=['categorical_crossentropy'],metrics=['categorical_crossentropy'],optimizer=Adam())
hjk_DL_model.fit([t_x,t_x_type],keras.utils.to_categorical(t_y), epochs=200,batch_size=3000,validation_data=[[v_x,v_x_type],keras.utils.to_categorical(v_y)],callbacks=[hjk_scher,es])

# Machine Learning

In [ ]:
def add_5_f(df :pd.DataFrame, i):
    df[str(i) + '_add_' +str(i+4)]=df.iloc[:,i] + df.iloc[:,i+1] + df.iloc[:,i+2] + df.iloc[:,i+3] + df.iloc[:,i+4]
    df[str(i) + '_mul_' +str(i+4)]=df.iloc[:,i] * df.iloc[:,i+1] * df.iloc[:,i+2] * df.iloc[:,i+3] * df.iloc[:,i+4]
    df[str(i) + '_add_1' +str(i+4)]=df.iloc[:,i] + df.iloc[:,i+1]
    df[str(i) + '_add_2' +str(i+4)]=df.iloc[:,i] + df.iloc[:,i+1]+df.iloc[:,i+2]
    df[str(i) + '_add_3' +str(i+4)]=df.iloc[:,i] + df.iloc[:,i+1]+df.iloc[:,i+2]+ df.iloc[:,i+3]
    df[str(i) + '_mul_1' +str(i+4)]=df.iloc[:,i] * df.iloc[:,i+1] 
    df[str(i) + '_mul_2' +str(i+4)]=df.iloc[:,i] * df.iloc[:,i+1] * df.iloc[:,i+2]
    df[str(i) + '_mul_3' +str(i+4)]=df.iloc[:,i] * df.iloc[:,i+1] * df.iloc[:,i+2]* df.iloc[:,i+3]
    df[str(i) + '_mean_' +str(i+4)]=np.mean([df.iloc[:,i] , df.iloc[:,i+1] , df.iloc[:,i+2] , df.iloc[:,i+3] , df.iloc[:,i+4]],axis=0)
    df[str(i) + '_stb_' +str(i+4)]=np.std([df.iloc[:,i] , df.iloc[:,i+1] , df.iloc[:,i+2] , df.iloc[:,i+3] , df.iloc[:,i+4]],axis=0)
    df[str(i) + '_max_' +str(i+4)]=np.max([df.iloc[:,i] , df.iloc[:,i+1] , df.iloc[:,i+2] , df.iloc[:,i+3] , df.iloc[:,i+4]],axis=0)
    df[str(i) + '_min_' +str(i+4)]=np.max([df.iloc[:,i] , df.iloc[:,i+1] , df.iloc[:,i+2] , df.iloc[:,i+3] , df.iloc[:,i+4]],axis=0)

def add_4_f(df :pd.DataFrame, i):
    df[str(i) + '_add_' +str(i+15)]=df.iloc[:,i] + df.iloc[:,i+5] + df.iloc[:,i+10] + df.iloc[:,i+15]
    df[str(i) + '_add_1' +str(i+15)]=df.iloc[:,i] + df.iloc[:,i+5] 
    df[str(i) + '_add_2' +str(i+15)]=df.iloc[:,i] + df.iloc[:,i+5] + df.iloc[:,i+10]
    df[str(i) + '_mul_' +str(i+15)]=df.iloc[:,i] * df.iloc[:,i+5] * df.iloc[:,i+10] * df.iloc[:,i+15]
    df[str(i) + '_mul_1' +str(i+15)]=df.iloc[:,i] * df.iloc[:,i+5]
    df[str(i) + '_mul_2' +str(i+15)]=df.iloc[:,i] * df.iloc[:,i+5]* df.iloc[:,i+10]
    df[str(i) + '_mean_' +str(i+15)]=np.mean([df.iloc[:,i] , df.iloc[:,i+5] , df.iloc[:,i+10] , df.iloc[:,i+15]],axis=0)
    df[str(i) + '_stb_' +str(i+15)]=np.std([df.iloc[:,i] , df.iloc[:,i+5] , df.iloc[:,i+10] , df.iloc[:,i+15]],axis=0)

def add_f(df :pd.DataFrame, i, j):
    df[str(i) + '_add_' +str(j)]=df.iloc[:,i] + df.iloc[:,j]
    df[str(i) + '_sub_' +str(j)]=np.abs(df.iloc[:,i] - df.iloc[:,j])
    df[str(i) + '_mul_' +str(j)]=df.iloc[:,i] * df.iloc[:,j]
    df[str(i) + '_dev_' +str(j)]=df.iloc[:,i] / (df.iloc[:,j] +1)
    df[str(j) + '_dev_' +str(i)]=df.iloc[:,j] / (df.iloc[:,i] +1)

def add_s_f(df :pd.DataFrame, i):
    df['sin' + str(i)]=np.sin(df.iloc[:,i])
    df['cos' + str(i)]=np.cos(df.iloc[:,i])
    df['tan' + str(i)]=np.tan(df.iloc[:,i])
    df['log' + str(i)]=np.log1p(np.abs(df.iloc[:,i]))
    df['inv'+ str(i)]=1/(df.iloc[:,i]+1)
    df['sqrt'+str(i)]=np.sqrt(df.iloc[:,i])
    df['square'+str(i)]=np.square(df.iloc[:,i])
    df['exp'+str(i)]=np.exp(df.iloc[:,i])
    df['tanh'+str(i)]=np.tanh(df.iloc[:,i])
    df['arccos'+str(i)]=np.arccos(df.iloc[:,i])

In [ ]:
train_X = train.iloc[:,3:]  #Train 데이터의 독립변수
train_X_type=train.iloc[:,2]
train_Y = train.iloc[:,1] #Train 데이터의 종속변수
test_X = test.iloc[:,2:]    #Test  데이터의 독립변수
test_X_type=test.iloc[:,1]
test_ids = test['id']


In [ ]:
for f in feature_list:
    min_value=train_X[f].min()
    train_X[f]=train_X[f].apply(lambda x : np.log1p(x-min_value))
    test_X[f]=test_X[f].apply(lambda x : np.log1p(x-min_value))

In [ ]:
train_X['f_id']=train_X_type
test_X['f_id']=test_X_type

In [ ]:
scaler = MinMaxScaler()
scaler.fit(train_X)
train_X=scaler.transform(train_X)
test_X=scaler.transform(test_X)

In [ ]:
train_X=pd.DataFrame(train_X)
test_X=pd.DataFrame(test_X)

In [ ]:
train_Y = train_Y.replace(label_dict)

In [ ]:
for i in range(20):
    add_s_f(train_X,i)
    add_f(train_X,i,20)
    add_s_f(test_X,i)
    add_f(test_X,i,20)

for i in range(4):
    add_f(train_X,(i*5),(i*5)+1)
    add_f(train_X,(i*5),(i*5)+2)
    add_f(train_X,(i*5),(i*5)+3)
    add_f(train_X,(i*5),(i*5)+4)
    add_f(train_X,(i*5)+1,(i*5)+2)
    add_f(train_X,(i*5)+1,(i*5)+3)
    add_f(train_X,(i*5)+1,(i*5)+4)
    add_f(train_X,(i*5)+2,(i*5)+3)
    add_f(train_X,(i*5)+2,(i*5)+4)
    add_f(train_X,(i*5)+3,(i*5)+4)
    
    add_f(test_X,(i*5),(i*5)+1)
    add_f(test_X,(i*5),(i*5)+2)
    add_f(test_X,(i*5),(i*5)+3)
    add_f(test_X,(i*5),(i*5)+4)
    add_f(test_X,(i*5)+1,(i*5)+2)
    add_f(test_X,(i*5)+1,(i*5)+3)
    add_f(test_X,(i*5)+1,(i*5)+4)
    add_f(test_X,(i*5)+2,(i*5)+3)
    add_f(test_X,(i*5)+2,(i*5)+4)
    add_f(test_X,(i*5)+3,(i*5)+4)

    add_5_f(train_X,(i*5))
    add_5_f(test_X,(i*5))

for i in range(5):
    add_f(train_X,i,i+5)
    add_f(train_X,i,i+10)
    add_f(train_X,i,i+15)
    add_f(train_X,i+5,i+10)
    add_f(train_X,i+5,i+15)
    add_f(train_X,i+10,i+15)

    add_f(test_X,i,i+5)
    add_f(test_X,i,i+10)
    add_f(test_X,i,i+15)
    add_f(test_X,i+5,i+10)
    add_f(test_X,i+5,i+15)
    add_f(test_X,i+10,i+15)

    add_4_f(train_X,(i))
    add_4_f(test_X,(i))

In [ ]:
print(train_X.min().min())
print(train_X.max().max())
t=train_X.isnull().sum()
t[t>0]

In [ ]:
t_x,v_x,t_x_type,v_x_type,t_y,v_y = train_test_split(train_X,train_X_type,train_Y , stratify = train_Y , random_state = 1000, test_size=0.2)


In [ ]:
def hjk_lr_sch(current_iter):
    init_lr=0.1
    min_lr=0.001
    e=1000
    step=np.exp(np.log(min_lr/init_lr)/e)
    if(current_iter<5):
        return init_lr
    else :
        hjk_lr=init_lr*step**(current_iter-5)
        if(hjk_lr>min_lr):
            return hjk_lr
        else:
            return min_lr

def hjk_lr_sch_2(current_iter):
    init_lr=0.07
    min_lr=0.03
    hjk_lr=init_lr*np.power(0.998,current_iter)
    if(hjk_lr>min_lr):
        return hjk_lr
    else:
        return min_lr


In [ ]:
import lightgbm
from lightgbm import LGBMClassifier

lgbm_clf = LGBMClassifier(
    boosting_type='gbdt',
    n_estimators=2000,
    learning_rate=0.1,
    num_leaves=160,
    max_depth=7,
    min_child_samples=20,
    min_child_weight=2,
    random_state=1,

    colsample_bytree=0.9,
    sub_sample=0.9,
    # subsample_freq=0.,
    reg_alpha=0.,
    reg_lambda=0.,
    importance_type='gain',
    tree_method='gpu_hist'
)

lgbm_clf.fit(t_x, t_y,eval_set=(v_x,v_y),early_stopping_rounds=20,callbacks=[lightgbm.reset_parameter(learning_rate=hjk_lr_sch_2)])

In [ ]:
pred_mat = hjk_model.predict(test_X)
sample = pd.read_csv('/gdrive/My Drive/SPACE/sample_submission.csv')
submission = pd.DataFrame(pred_mat, index=test.index)
submission = submission.rename(columns=i2lb)
submission = pd.concat([test_ids, submission], axis=1)
submission = submission[sample.columns]
submission.to_csv("/gdrive/My Drive/SPACE/hjk_space_10_2.csv", index=False)